# PyUnfold Tutorial

This tutorial can be launched on an interactive notebook server using [Binder](https://mybinder.org/). You can open a notebook server [here](https://mybinder.org/v2/gh/jrbourbeau/pyunfold/master?filepath=docs%2Fsource%2Fnotebooks%2Ftutorial.ipynb). In addition, note that if you wish to run this notebook on your own, you'll need to have `matplotlib` installed. 

PyUnfold uses the `iterative_unfold` function to perform iterative unfoldings. In addition, we'll use the `Logger` callback (see the [Callbacks documentation](../callbacks.rst) for more information) to display the unfolding status at each iteration. 

In [ ]:
from pyunfold import iterative_unfold
from pyunfold.callbacks import Logger

In [ ]:
import numpy as np
np.random.seed(2)

import matplotlib as mpl
mpl.rcParams['font.size'] = 16.0
mpl.rcParams['axes.labelsize'] = 16.0
mpl.rcParams['axes.titlesize'] = 14.0
mpl.rcParams['legend.fontsize'] = 12.0

import matplotlib.pyplot as plt 

%matplotlib inline

## True distribution

Before we can perform an unfolding, we'll need some true and observed data. In this tutorial, we'll use the `numpy.random` module to generate some example data. Let's assume that we are measuring some observable, $X$, that is given by a normal (Gaussian) distribution.

In [ ]:
num_samples = int(1e5)
true_samples = np.random.normal(loc=0.0, scale=1.0, size=num_samples)
true_samples

These samples can then be binned to form a histogram of the true distribution.

In [ ]:
bins = np.linspace(-3, 3, 21)
num_bins = len(bins) - 1

In [ ]:
data_true, _ = np.histogram(true_samples, bins=bins)
data_true

The true distribution of $X$ looks like 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.step(np.arange(num_bins), data_true, where='mid', lw=3,
        alpha=0.7, label='True distribution')
ax.set(xlabel='X bins', ylabel='Counts')
ax.legend()
plt.show()

## Observed distribution

Generally speaking, measurement devices aren't perfect. This means that the measured value of $X$ isn't always exactly the same as the true value. That is, our measurement device has some inherit bias and resolution. 

We'll model these detection imperfections by adding some random Gaussian noise to our true values of $X$ to get our observed (measured) values. 

In [ ]:
random_noise = np.random.normal(loc=0.3, scale=0.5, size=num_samples)
observed_samples = true_samples + random_noise

The observed values can be binned into a histogram. This array, `data_observed`, is the distribution we want to unfold.  

In [ ]:
data_observed, _ = np.histogram(observed_samples, bins=bins)
data_observed

We can see how the true and observed distribution of $X$ differ. 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.step(np.arange(num_bins), data_true, where='mid', lw=3,
        alpha=0.7, label='True distribution')
ax.step(np.arange(num_bins), data_observed, where='mid', lw=3,
        alpha=0.7, label='Observed distribution')
ax.set(xlabel='X bins', ylabel='Counts')
ax.legend()
plt.show()

In addition to the observations themselves, the uncertainty of the observations is also needed to perform an unfolding. Here, we'll assume the errors are simply Poisson counting errors (i.e. $\mathrm{error_{i} = \sqrt{counts_{i}}}$).

In [ ]:
data_observed_err = np.sqrt(data_observed)
data_observed_err

## Detection Efficiencies

Not every sample from our true distribution will lead to a measured observation. The probability that a sample from our true distribution is observed is quantified via our detection efficiencies. For now, we'll assume uniform efficiencies of 1.

In [ ]:
efficiencies = np.ones_like(data_observed, dtype=float)
efficiencies

Likewise, let's assume an uncertainty on our efficiencies of 0.1. 

In [ ]:
efficiencies_err = np.full_like(efficiencies, 0.1, dtype=float)
efficiencies_err

## Response matrix

The next step is to construct a response matrix that encapsulates our detector bias and resolution. This can be done by making a 2-dimensional histogram of the true vs. observed distributions.  

In [ ]:
response_hist, _, _ = np.histogram2d(observed_samples, true_samples, bins=bins)

As with the uncertainties on our observed data, we'll assume Poisson counting errors for our response matrix. 

In [ ]:
response_hist_err = np.sqrt(response_hist)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(response_hist, origin='lower')
cbar = plt.colorbar(im, label='Counts')
ax.set(xlabel='Cause bins', ylabel='Effect bins')
plt.show()

However, there's still one more step for the response matrix. What's needed for unfolding is not this 2-d counts histogram, but a matrix with the conditional probabilities $P(E_i|C_j)$. This matrix, called the normalized response matrix, can be obtained by normalizing our 2-d counts matrix. That is, scale each column of `response_hist` such that it adds to our detection efficiencies.

This normalization factor can be easily found via:

In [ ]:
column_sums = response_hist.sum(axis=0) 
normalization_factor = efficiencies / column_sums

In [ ]:
response = response_hist * normalization_factor
response_err = response_hist_err * normalization_factor

As a check we can see that the columns of `response` add to our detection efficiencies. 

In [ ]:
response.sum(axis=0)

We can see how the structure of the 2-d response histogram differs from the normalized response matrix. 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(response, origin='lower')
cbar = plt.colorbar(im, label='$P(E_i|C_{\mu})$')
ax.set(xlabel='Cause bins', ylabel='Effect bins',
       title='Normalizes response matrix')
plt.show()

This difference in structure is because each column in the normalized response matrix is scaled by the number of samples in that column. So only the _relative_ structure of each column of the 2-d response histogram is present in the normalized response matrix. 

## Iterative unfolding

Now we have everything we need to perform our unfolding:

- Observed distribution to unfold (with uncertainties): `data_observed`, `data_observed_err`
- Detection efficiencies (with uncertainties): `efficiencies`, `efficiencies_err`
- Normalized response matrix (with uncertainties): `response`, `response_err`

These are all input parameters into the PyUnfold `iterative_unfold` function.

In [ ]:
unfolded_results = iterative_unfold(data=data_observed,
                                    data_err=data_observed_err,
                                    response=response,
                                    response_err=response_err,
                                    efficiencies=efficiencies,
                                    efficiencies_err=efficiencies_err,
                                    callbacks=[Logger()])

What's returned from `iterative_unfold` is a dictionary that contains:

- Unfolded distribution (`unfolded` key)
- Statistical uncertainties on the unfolded distribution (`stat_err` key)
- Systematic uncertainties on the unfolded distribution based on the response matrix uncertainties (`sys_err` key)
- Number of unfolding iterations to reach test statistic stopping condition (`num_iterations` key)
- Test statistic value for final iteration (`ts_iter` key)
- Test statistic stopping condition (`ts_stopping` key)

In [ ]:
unfolded_results

Finally, we can see how the true, observed, and unfolded distributions compare to one another. 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.step(np.arange(num_bins), data_true, where='mid', lw=3,
        alpha=0.7, label='True distribution')
ax.step(np.arange(num_bins), data_observed, where='mid', lw=3,
        alpha=0.7, label='Observed distribution')
ax.errorbar(np.arange(num_bins), unfolded_results['unfolded'],
            yerr=unfolded_results['sys_err'],
            alpha=0.7,
            elinewidth=3,
            capsize=4,
            ls='None', marker='.', ms=10, 
            label='Unfolded distribution')

ax.set(xlabel='X bins', ylabel='Counts')
plt.legend()
plt.show()

Note that the unfolded distribution is consistent with the true distribution! 